In [ ]:
deploy_phase = "local"
execution_date = '2024-07-06T00:00:00+09:00'
content_index = 0

# Notebook Initialization

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import sys
# sys.path.insert(0, "/workspace/ojitong")

# Global Const

In [ ]:
from datetime import datetime

def convert_to_datetime(date_str: str) -> datetime:
    return datetime.fromisoformat(date_str)

today_datetime = convert_to_datetime(execution_date)
today_datetime

In [ ]:
import pandas as pd
from data.utils.file import generate_s3_path
s3_path = generate_s3_path(prefix="news_data_selected", today_datetime=today_datetime)
df = pd.read_csv("/workspace/ojitong/output_0706.csv")

In [ ]:
content_row = df.iloc[content_index]

In [ ]:
# 1. 필요한 데이터 추출
title = content_row['title']
publish_date = content_row['publish_date']
article = content_row['text']
category = content_row['category']

In [ ]:
# 출력
print("Title:", title)
print("Publish Date:", publish_date)
print("Article:", article)
print("Category:", category)

In [ ]:
# 2. source content 생성
publish_date = datetime.strptime(publish_date, '%Y-%m-%d %H:%M:%S%z')
formatted_date_str = publish_date.strftime('%Y-%m-%d %H:%M:%S')
source_content = title + formatted_date_str + article

In [ ]:
from data.tasks.writing_task import ChainingTask
task = ChainingTask(category=category)

In [ ]:
response = task.execute(source_content=source_content)
response

In [ ]:
# SystemMessage 객체 리스트를 문자열로 변환하는 함수
def extract_chat_history(messages):
    return '\n\n'.join(message.content for message in messages)

In [ ]:
blog_contents = {
    "question": response["question"],
    "chat_history": extract_chat_history(response["chat_history"]),
    "blog_content": response["text"],
}

In [ ]:
blog_contents

In [ ]:
# 기존 DF
for key, value in blog_contents.items():
    df.at[content_index, key] = value

In [ ]:
# 3. 업데이트 된 파일 S3 업로드
s3_path = generate_s3_path(prefix="blog_contents", today_datetime=today_datetime)
df.to_csv(s3_path, index=False)
print(f"*** Data saved to {s3_path} ***")